## Section 1: Import Libraries and load cifar10 dataset.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
#import additional libraries if needed

In [2]:
import os

def load_cifar10_data(batch_size):
  transform = transforms.Compose(
      [transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(
          mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
      )])
    
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
  
  trainset = torch.utils.data.Subset(trainset, range(10000))

  cpu_cores = os.cpu_count()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=cpu_cores // 2, pin_memory=True)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
  
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=cpu_cores // 2, pin_memory=True)
  
  return trainloader, testloader, trainset

In [3]:
batch_size = 32
trainloader, testloader, trainset = load_cifar10_data(batch_size)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Check the dimensions of a batch:
for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([32, 3, 224, 224])
Image label dimensions: torch.Size([32])


## Section 2: Load Pre-trained Model 

In [5]:
from torchvision import models as torchvision_models
model_list = []

# AlexNet
alexnet = torchvision_models.alexnet(pretrained=True)
alexnet.name = 'Alexnet'
model_list.append(alexnet)

# VGG16
VGG16 = torchvision_models.vgg16(pretrained=True)
VGG16.name = 'VGG16'
model_list.append(VGG16)

# ResNet-50
resnet = torchvision_models.resnet50(pretrained=True)
resnet.name = 'Resnet-50'
model_list.append(resnet)

for model in model_list:
    print(f"{model.name} loaded.")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET

Alexnet loaded.
VGG16 loaded.
Resnet-50 loaded.


In [6]:
import torch.nn as nn
import torch.optim as optim

for param in alexnet.parameters():
    param.requires_grad = False

# Alex-net
output_layer_index = 6
in_features = alexnet.classifier[output_layer_index].in_features
no_of_cifar_classes = 10
alexnet.classifier[output_layer_index] = nn.Linear(in_features, no_of_cifar_classes)
for param in alexnet.classifier[output_layer_index].parameters():
    param.requires_grad = True

# VGG16
output_layer_index = 6
in_features = VGG16.classifier[output_layer_index].in_features
no_of_cifar_classes = 10
VGG16.classifier[output_layer_index] = nn.Linear(in_features, no_of_cifar_classes)
for param in VGG16.classifier[output_layer_index].parameters():
    param.requires_grad = True

# Resnet-50
in_features = resnet.fc.in_features
no_of_cifar_classes = 10
resnet.fc = nn.Linear(in_features, no_of_cifar_classes)
for param in resnet.fc.parameters():
    param.requires_grad = True

## Section 3: Define optimizer and loss criterion

In [7]:
#TODO: Declare your hyperparameters here
random_seed = 42
learning_rate = 0.001
num_epochs = 8
num_folds = 10

np.random.seed(random_seed)
torch.manual_seed(random_seed)

optimizers = {}
for model in model_list:
    optimizers[model.name] = optim.SGD(model.parameters(), lr=learning_rate * (batch_size / 4), momentum=0.9, weight_decay=1e-4)
    
criterion = nn.CrossEntropyLoss()
foldLen = len(trainset)//num_folds

In [8]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)

Device: cuda:0


## Section 4: Train the model

In [9]:
def train_model(model, num_epochs, trainloader, testloader, optimizer, criterion):
    epoch_accuracies = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        truePredictions = 0
        totalPredictions = 0
        
        model.train()
        for i, (features, labels) in enumerate(trainloader):
            
            #TODO: Create a training pipeline - compute the model logits, backward prop, and update step.
            inputs, labels = features.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)
            totalPredictions += labels.size(0)
            truePredictions += (predicted == labels).sum().item()
            
            #TODO: Print the accuracy and loss as well.
            running_loss += loss.item()
            
        epoch_loss = running_loss / len(trainloader)
        epoch_acc = 100 * truePredictions / totalPredictions
        epoch_accuracies.append(epoch_acc)
        print(f"Epoch Number [{epoch + 1}/{num_epochs}], Loss Value: {epoch_loss:.4f}, Accuracy this fold: {epoch_acc:.2f}%")
        
    model.eval()
    with torch.no_grad():
        truePredictions = 0
        totalPredictions = 0
        for (features, labels) in testloader:
            images, labels = features.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            totalPredictions += labels.size(0)
            truePredictions += (predicted == labels).sum().item()

        validation_accuracy = 100 * truePredictions / totalPredictions
        print(f"Accuracy for the fold on test data: {validation_accuracy:.2f}%\n")       
        
    train_accuracy = sum(epoch_accuracies) / num_epochs
        
    return train_accuracy, validation_accuracy    

## Section 5: Test the model

In [10]:
for model in model_list:
  print(model.name)
  # optimizer
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate * (batch_size / 4), momentum=0.9, weight_decay=1e-4)

  DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print('Device:', DEVICE)
  model = model.to(DEVICE)

  test_accuracies = []
  train_accuracies = []
  for fold in range(num_folds):
      print(f'Fold-- {fold+1}')
      test_start_index = foldLen * fold
      test_end_index = test_start_index + foldLen
      
      train_indices = list(range(0, test_start_index)) + list(range(test_end_index, len(trainset)))
      test_indices = list(range(test_start_index, test_end_index))
      
      train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
      validation_sampler = torch.utils.data.sampler.SubsetRandomSampler(test_indices)

      cpu_cores = os.cpu_count()

      train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=train_sampler, num_workers=cpu_cores//2, pin_memory=True)
      validation_loader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=validation_sampler, num_workers=cpu_cores//2, pin_memory=True)

      train_accuracy, validation_accuracy = train_model(model, num_epochs, train_loader, validation_loader, optimizer, criterion)
      test_accuracies.append(validation_accuracy)
      train_accuracies.append(train_accuracy)
      
  avg_validation_accuracy = sum(test_accuracies) / num_folds
  avg_train_accuracy = sum(train_accuracies) / num_folds
  print(f"Accuracy of the network on the train images for {model.name}: {avg_train_accuracy:.2f}%")
  print(f"Accuracy of the network on the 10000 test images for {model.name}: {avg_validation_accuracy:.2f}%")
  print(f"Cross validation done for {model.name}")

Alexnet
Device: cuda:0
Fold-- 1
Epoch Number [1/8], Loss Value: 2.2835, Accuracy this fold: 61.18%
Epoch Number [2/8], Loss Value: 1.8299, Accuracy this fold: 69.79%
Epoch Number [3/8], Loss Value: 1.8217, Accuracy this fold: 70.37%
Epoch Number [4/8], Loss Value: 1.6206, Accuracy this fold: 73.53%
Epoch Number [5/8], Loss Value: 1.5973, Accuracy this fold: 73.54%
Epoch Number [6/8], Loss Value: 1.6333, Accuracy this fold: 73.51%
Epoch Number [7/8], Loss Value: 1.6066, Accuracy this fold: 74.02%
Epoch Number [8/8], Loss Value: 1.5283, Accuracy this fold: 75.48%
Accuracy for the fold on test data: 71.70%

Fold-- 2
Epoch Number [1/8], Loss Value: 1.6222, Accuracy this fold: 74.98%
Epoch Number [2/8], Loss Value: 1.8187, Accuracy this fold: 73.97%
Epoch Number [3/8], Loss Value: 1.6270, Accuracy this fold: 74.86%
Epoch Number [4/8], Loss Value: 1.6727, Accuracy this fold: 74.92%
Epoch Number [5/8], Loss Value: 1.5699, Accuracy this fold: 76.27%
Epoch Number [6/8], Loss Value: 1.5011, Accu